# 911 Calls Capstone Project

# Data and Setup

____
** Import numpy and pandas **

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

** Import visualization libraries and set %matplotlib inline. **

In [ ]:
%matplotlib inline

** Read in the csv file as a dataframe called df **

In [ ]:
df=pd.read_csv('../input/911.csv')

** Check the info() of the df **

In [ ]:
df.info()

** Check the head of df **

In [ ]:
df.head()

# Basic Questions

** What are the top 5 zipcodes for 911 calls? **

In [ ]:
df['zip'].value_counts().head(5)

** What are the top 5 townships (twp) for 911 calls? **

In [ ]:
df['twp'].value_counts().head(5)

** Take a look at the 'title' column, how many unique title codes are there? **

In [ ]:
df['title'].nunique()

# Creating new features

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.** 

**For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. **

In [ ]:
df['reason'] = df['title'].apply(lambda t: t.split(':')[0])

** What is the most common Reason for a 911 call based off of this new column? **

In [ ]:
df['reason'].value_counts()

** Now use seaborn to create a countplot of 911 calls by Reason. **

In [ ]:
sns.countplot(x='reason', data=df, palette='viridis')

___
** Now let us begin to focus on time information. What is the data type of the objects in the timeStamp column? **

In [ ]:
type(df['timeStamp'][0])

** You should have seen that these timestamps are still strings. Use [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to convert the column from strings to DateTime objects. **

In [ ]:
df['timeStamp'] = df['timeStamp'].apply(pd.to_datetime)

** You can now grab specific attributes from a Datetime object by calling them. For example:**

    time = df['timeStamp'].iloc[0]
    time.hour

**You can use Jupyter's tab method to explore the various attributes you can call. Now that the timestamp column are actually DateTime objects, use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column, reference the solutions if you get stuck on this step.**

In [ ]:
df['hour'] = df['timeStamp'].apply(lambda dt: dt.hour)
df['dayOfWeek'] = df['timeStamp'].apply(lambda dt: dt.strftime('%a'))
df['month'] = df['timeStamp'].apply(lambda dt: dt.month)

In [ ]:
df.head()

** Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week: **

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

** Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. **

In [ ]:
## Display in spreadsheet format
df.groupby(['dayOfWeek','reason']).count()['e']

In [ ]:
## Display in Pivot Table format
df.pivot_table(index='dayOfWeek', columns='reason', values='e', aggfunc='count')

In [ ]:
## Visualised Display
sns.countplot(x='dayOfWeek', hue='reason', data=df, palette='viridis')
plt.legend(bbox_to_anchor=(1.02,0.72), loc=3)
plt.xlabel('day')

**Now do the same for Month:**

In [ ]:
sns.countplot(x='month', data=df, hue='reason', palette='viridis')
plt.legend(bbox_to_anchor=(1.02,0.72), loc=3)
plt.xlabel('Month')

**Did you notice something strange about the Plot?**

_____

** You should have noticed it was missing some Months, let's see if we can maybe fill in this information by plotting the information in another way, possibly a simple line plot that fills in the missing months, in order to do this, we'll need to do some work with pandas... **

** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame. **

In [ ]:
byMonth = df.groupby('month').count()
byMonth

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
byMonth['e'].plot()

** Create a plot of the dataframe with 3 types of reasons **

In [ ]:
byMonthReason = df.pivot_table(index='month', columns='reason', aggfunc='count')['e']

In [ ]:
byMonthReason.plot()

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
sns.lmplot(x='month', y='twp', data=byMonth.reset_index())

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method. ** 

In [ ]:
df['date'] = df['timeStamp'].apply(datetime.date)

In [ ]:
datetime.date(df['timeStamp'][0])

** Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
df.groupby('date')['e'].count().plot()
plt.tight_layout()
plt.xlabel('Date')

** Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
df[df['reason'] == 'Traffic'].groupby('date')['e'].count().plot()
plt.xlabel('Date')
plt.title('Traffic')
plt.tight_layout()

In [ ]:
df[df['reason'] == 'Fire'].groupby('date')['e'].count().plot()
plt.title('Fire')
plt.tight_layout()

In [ ]:
df[df['reason'] == 'EMS'].groupby('date')['e'].count().plot()
plt.title('EMS')
plt.tight_layout()

____
** Now let's move on to creating  heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an [unstack](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method. Reference the solutions if you get stuck on this!**

In [ ]:
dayHour = df.pivot_table(index='dayOfWeek', columns='hour', values='e', aggfunc='count')

## Or Using 'groupby + unstack':
## dayHour = df.groupby(by=['dayOfWeek','hour']).count()['e'].unstack(level=-1)

## Reorder index by weekdays
dayList = 'Mon Tue Wed Thu Fri Sat Sun'.split()
dayHour.index = pd.Categorical(dayHour.index, dayList, ordered=True)
dayHour.index.name = 'dayOfWeek'

dayHour.sort_index(inplace=True)

** Now create a HeatMap using this new DataFrame. **

In [ ]:
sns.heatmap(data=dayHour, cmap='viridis')

** Now create a clustermap using this DataFrame. **

In [ ]:
sns.clustermap(data=dayHour, cmap='viridis')

** Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. **

In [ ]:
dayMonth = df.groupby(['dayOfWeek','month']).count()['e'].unstack(level=1)
dayMonth.index = pd.Categorical(dayMonth.index, categories=dayList, ordered=True)
dayMonth.index.name = 'dayOfWeek'

## Reorder dayMonth index by weekdays
dayMonth.sort_index(inplace=True)
dayMonth

In [ ]:
sns.heatmap(dayMonth, cmap='viridis')

In [ ]:
sns.clustermap(dayMonth, cmap='viridis')